In [2]:
import pandas as pd
from database.market import Market
import matplotlib.pyplot as plt
from datetime import datetime
from transformer.date_transformer import DateTransformer
from transformer.column_transformer import ColumnTransformer
import math
from tqdm import tqdm

In [3]:
market = Market()

In [4]:
market.connect()
sp5 = market.retrieve_data("sp500")
market.close()

In [7]:
analysis = []
prices = []
fails = []
datasets = ["pdr","finnhub","tiingo"]
for ticker in tqdm(sp5["Symbol"]):
    try:
        market.connect()
        pdr = market.retrieve_price_data("pdr_prices",ticker)
        finnhub = market.retrieve_price_data("finnhub_prices",ticker)
        tiingo = market.retrieve_price_data("tiingo_prices",ticker)
        pdr = DateTransformer.convert_to_date("pdr",ColumnTransformer.rename_columns(pdr," "),"date")
        finnhub = DateTransformer.convert_to_date("finnhub",ColumnTransformer.rename_columns(finnhub,"_"),"date")
        tiingo = DateTransformer.convert_to_date("tiingo",ColumnTransformer.rename_columns(tiingo,"_"),"date")
        market.close()
        tiingo["date"] = [str(x).split(" ")[0] for x in tiingo["date"]]
        finnhub["date"] = [str(x).split(" ")[0] for x in finnhub["date"]]
        pdr["date"] = [str(x).split(" ")[0] for x in pdr["date"]]
        pdr.rename(columns={"adjclose":"pdr"},inplace=True)
        finnhub.rename(columns={"adjclose":"finnhub"},inplace=True)
        tiingo.rename(columns={"adjclose":"tiingo"},inplace=True)
        ds = pdr.merge(finnhub,on="date",how="left").merge(tiingo,on="date",how="left")
        ds["pf"] = (ds["pdr"]  - ds["finnhub"]) / ds["pdr"]
        ds["pt"] = (ds["pdr"]  - ds["tiingo"]) / ds["pdr"]
        ds["ft"] = (ds["finnhub"]  - ds["tiingo"]) / ds["finnhub"]
        prices.append(ds)
        analysis_dict = {"ticker":ticker}
        for col in ["pf","pt","ft"]:
            ds[col] = [abs(x) for x in ds[col]]
            analysis_dict[col] = ds[col].mean()
        analysis.append(analysis_dict)
    except:
        fails.append(ticker)
        continue

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 506/506 [01:59<00:00,  4.25it/s]


In [8]:
a = pd.DataFrame(analysis)

In [52]:
a["diff"] =[row[1][["pf","pt","ft"]].mean() for row in a.iterrows()]
accurate = a[(a["diff"]<0.01) & (a["pt"]<0.1) & (a["pf"]<0.1) & (a["ft"]<0.1)].sort_values("diff")

In [54]:
market.connect()
market.store_data("accurate",accurate)
market.close()